# Hyperoptimization with optuna

## 1. Dataset

Example for dataset ESOL from Moleculenet.

In [1]:
import numpy as np
import keras as ks

In [2]:
from kgcnn.data.datasets.ESOLDataset import ESOLDataset
dataset = ESOLDataset()
dataset.set_attributes()
dataset.map_list(method="set_edge_indices_reverse");
dataset.map_list(method="count_nodes_and_edges");
dataset.map_list(**{"method": "count_nodes_and_edges", "total_edges": "total_reverse"});

INFO:kgcnn.data.download:Checking and possibly downloading dataset with name ESOL
INFO:kgcnn.data.download:Dataset directory located at C:\Users\patri\.kgcnn\datasets
INFO:kgcnn.data.download:Dataset directory found. Done.
INFO:kgcnn.data.download:Dataset found. Done.
INFO:kgcnn.data.ESOL:Found SDF C:\Users\patri\.kgcnn\datasets\ESOL\delaney-processed.sdf of pre-computed structures.
INFO:kgcnn.data.ESOL:Read molecules from mol-file.
INFO:kgcnn.data.ESOL: ... process molecules 0 from 1128
INFO:kgcnn.molecule.encoder:OneHotEncoder Symbol found ['O', 'C', 'N', 'S', 'Cl', 'P', 'F', 'I', 'Br']
INFO:kgcnn.molecule.encoder:OneHotEncoder Hybridization found [rdkit.Chem.rdchem.HybridizationType.SP3, rdkit.Chem.rdchem.HybridizationType.SP, rdkit.Chem.rdchem.HybridizationType.SP2]
INFO:kgcnn.molecule.encoder:OneHotEncoder TotalDegree found [2, 4, 1, 3]
INFO:kgcnn.molecule.encoder:OneHotEncoder TotalNumHs found [1, 2, 0, 3, 4]
INFO:kgcnn.molecule.encoder:OneHotEncoder CIPCode found [None, 'S', 'R'

## 2. Model

Pick DMPNN as example model.

In [3]:
from kgcnn.literature.DMPNN import make_model

In [5]:
model_config = {
    "name": "DMPNN",
    "inputs": [
        {"shape": (None, 41), "name": "node_attributes", "dtype": "float32"},
        {"shape": (None, 11), "name": "edge_attributes", "dtype": "float32"},
        {"shape": (None, 2), "name": "edge_indices", "dtype": "int64"},
        {"shape": (None, 1), "name": "edge_indices_reverse", "dtype": "int64"},
        {"shape": (), "name": "total_nodes", "dtype": "int64"},
        {"shape": (), "name": "total_edges", "dtype": "int64"},
        {"shape": (), "name": "total_reverse", "dtype": "int64"}
    ],
    "cast_disjoint_kwargs": {},
    "input_node_embedding": {"input_dim": 95, "output_dim": 64},
    "input_edge_embedding": {"input_dim": 5, "output_dim": 64},
    "input_graph_embedding": {"input_dim": 100, "output_dim": 64},
    "pooling_args": {"pooling_method": "scatter_sum"},
    "edge_initialize": {"units": 128, "use_bias": True, "activation": "relu"},
    "edge_dense": {"units": 128, "use_bias": True, "activation": "linear"},
    "edge_activation": {"activation": "relu"},
    "node_dense": {"units": 128, "use_bias": True, "activation": "relu"},
    "verbose": 10, "depth": 5,
    "dropout": {"rate": 0.1},
    "output_embedding": "graph",
    "output_mlp": {
        "use_bias": [True, True, False], "units": [64, 32, 1],
        "activation": ["relu", "relu", "linear"]
    }
}
# Test making model
model = make_model(**model_config)

INFO:kgcnn.models.utils:Updated model kwargs: '{'name': 'DMPNN', 'inputs': [{'shape': (None, 41), 'name': 'node_attributes', 'dtype': 'float32'}, {'shape': (None, 11), 'name': 'edge_attributes', 'dtype': 'float32'}, {'shape': (None, 2), 'name': 'edge_indices', 'dtype': 'int64'}, {'shape': (None, 1), 'name': 'edge_indices_reverse', 'dtype': 'int64'}, {'shape': (), 'name': 'total_nodes', 'dtype': 'int64'}, {'shape': (), 'name': 'total_edges', 'dtype': 'int64'}, {'shape': (), 'name': 'total_reverse', 'dtype': 'int64'}], 'input_tensor_type': 'padded', 'cast_disjoint_kwargs': {}, 'input_embedding': None, 'input_node_embedding': {'input_dim': 95, 'output_dim': 64}, 'input_edge_embedding': {'input_dim': 5, 'output_dim': 64}, 'input_graph_embedding': {'input_dim': 100, 'output_dim': 64}, 'pooling_args': {'pooling_method': 'scatter_sum'}, 'use_graph_state': False, 'edge_initialize': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'edge_dense': {'units': 128, 'use_bias': True, 'activatio

In [6]:
dataset.clean(model_config["inputs"])

INFO:kgcnn.data.ESOL:Property 'edge_attributes' is an empty list for graph '934'.
INFO:kgcnn.data.ESOL:Property 'edge_indices' is an empty list for graph '934'.
INFO:kgcnn.data.ESOL:Property 'edge_indices_reverse' is an empty list for graph '934'.


array([934])

In [7]:
labels = np.expand_dims(dataset.get("graph_labels"), axis=-1)
labels.shape

(1127, 1)

## 3. Optuna

Loading optuna for hyperoptimization example. Example is modified from optuna [github](https://github.com/optuna/optuna-examples/blob/main/keras/keras_simple.py>) or web page https://optuna.org/#code_examples.

For this example we neglect target standardization or more metric options.

In [8]:
import warnings
import optuna
from copy import deepcopy
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.backend import clear_session

In [9]:
BATCHSIZE = 128
EPOCHS = 100

In [10]:
# Define objective function
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    train_index, test_index = train_test_split(np.arange(len(dataset)), test_size=0.25, random_state=42)
    x_train, y_train = dataset[train_index].tensor(model_config["inputs"]), labels[train_index]
    x_valid, y_valid = dataset[test_index].tensor(model_config["inputs"]), labels[test_index]
    
    hyper_trial = deepcopy(model_config)
    hyper_trial["depth"] = trial.suggest_int("depth", 1, 6, 2)
    hyper_trial["pooling_args"]["pooling_method"] = trial.suggest_categorical("pooling_method", ["scatter_sum", "scatter_mean"])
    units = trial.suggest_int("units", 32, 256, 32)
    hyper_trial["node_dense"]["units"] = units
    hyper_trial["edge_dense"]["units"] = units
    hyper_trial["edge_initialize"]["units"] = units
    # print(hyper_trial)
    
    model = make_model(**hyper_trial)
    
    # We compile our model
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate),
        metrics=["mean_absolute_error"],
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]

### Optimization

In [11]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=600)

[I 2023-11-30 16:26:26,453] A new study created in memory with name: no-name-c285b9ee-79cd-4488-b161-a254d0ef2c12
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.models.utils:Updated model kwargs: '{'name': 'DMPNN', 'inputs': [{'shape': (None, 41), 'name': 'node_attributes', 'dtype': 'float32'}, {'shape': (None, 11), 'name': 'edge_attributes', 'dtype': 'float32'}, {'shape': (None, 2), 'name': 'edge_indices', 'dtype': 'int64'}, {'shape': (None, 1), 'name': 'edge_indices_reverse', 'dtype': 'int64'}, {'shape': (), 'name': 'total_nodes', 'dtype': 'int64'}, {'shape': (), 'name': 'total_edges', 'dtype': 'int64'}, {'shape': (), 'name': 'total_reverse', 'dtype': 'int64'}], 'input_tensor_type': 'padded', 'cast_disjoint_kwargs': {}, 'input_embedding': None, 'input_node_embedding': {'input

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_1/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_1/Reshape:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_1/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed/gather_nodes_outgoing_1/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed/gather_nodes_outgoing_1/Reshape:0", shape=(None, 192), 

[I 2023-02-09 15:57:27,704] Trial 7 finished with value: 0.7500041127204895 and parameters: {'depth': 3, 'pooling_method': 'mean', 'units': 256, 'learning_rate': 6.234024612863678e-05}. Best is trial 1 with value: 0.4369063079357147.
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.model.utils:Updated model kwargs:
INFO:kgcnn.model.utils:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_3/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_3/Reshape:0", shape=(None, 256), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_3/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_2/gather_nodes_outgoing_3/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_2/gather_nodes_outgoing_3/Reshape:0", shape=(None, 25

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed/gather_nodes_outgoing_1/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed/gather_nodes_outgoing_1/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed/gather_nodes_outgoing_1/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed/dmpnn_gather_edges_pairs/gather_nodes_ingoing/Reshape_1:0", shape=(None,), dtype=int64), va

...

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 160), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 16

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 96), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 96)

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 12

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 96), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 96)

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Reshape:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/DMPNN/pooling_local_edges_5/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/DMPNN/dmpnnp_pooling_edges_directed_4/gather_nodes_outgoing_5/Reshape:0", shape=(None, 19

In [12]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 11
Best trial:
  Value: 0.5147700309753418
  Params: 
    depth: 3
    pooling_method: scatter_mean
    units: 192
    learning_rate: 0.0120211744680511
